# Proyecto 1 — Estación de llenado y taponado



Importación de las librerias

In [25]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

Definir las rutas de los archivos

In [26]:
data_path = Path("data")

telemetria_file = data_path / "telemetria.csv"
eventos_file = data_path / "eventos.csv"
botellas_file = data_path / "botellas.csv"

# FASE 1 --->Ingesta y validación (Pandas)


PASO 1: Carga y tipado de datos

    1.1 Definir tipos de datos explícitos para cada CSV utilizando un diccionario 

In [27]:

dtype_tel = {
    'temp_prod': 'float32',
    'vel_cinta': 'float32',
    'caudal': 'float32',
    'energia_kwh': 'float64'
}

dtype_evt = {
    'tipo': 'category',
    'id_botella': 'Int64'
}

dtype_pz = {
    'id_botella': 'int64',
    'peso_neto': 'float32',
    'formato': 'category'
}


    1.2 Cargar los csv en los dataFrames parseando el tiempo a datetime64

In [28]:
# Cargar telemetría con parseo de fecha
df_tel = pd.read_csv(
    telemetria_file,
    dtype=dtype_tel,
    parse_dates=['ts'],
    date_format='ISO8601'
)
# Convertir ts a UTC y establecer como índice
df_tel['ts'] = pd.to_datetime(df_tel['ts'], utc=True) # Convierte a datetime con zona horaria UTC
df_tel = df_tel.set_index('ts').sort_index() # Establece la columna de tiempo como índice del DataFrame

# Cargar eventos
df_evt = pd.read_csv(
    eventos_file,
    dtype=dtype_evt,
    parse_dates=['ts_ini', 'ts_fin'],
    date_format='ISO8601'
) # Lee el CSV y convierte automáticamente las columnas de fecha
df_evt['ts_ini'] = pd.to_datetime(df_evt['ts_ini'], utc=True) #Convierte a datetime con zona horaria UTC
df_evt['ts_fin'] = pd.to_datetime(df_evt['ts_fin'], utc=True) #Convierte a datetime con zona horaria UTC
df_evt = df_evt.sort_values('ts_ini').reset_index(drop=True)

# Cargar botellas
df_pz = pd.read_csv(
    botellas_file,
    dtype=dtype_pz,
    parse_dates=['ts_ciclo'],
    date_format='ISO8601'
)
df_pz['ts_ciclo'] = pd.to_datetime(df_pz['ts_ciclo'], utc=True) #Convierte a datetime con zona horaria UTC
df_pz = df_pz.sort_values('ts_ciclo').reset_index(drop=True) #


PASO 2. Orden y duplicados

    2.1 Telemetría

In [29]:
print("Telemetria:")
# Eliminar duplicados exactos en telemetría
duplicados_antes_tel = df_tel.duplicated().sum()
df_tel = df_tel[~df_tel.duplicated(keep='first')]

#df_evt = df_evt[~df_evt.duplicated(keep='first')]

# Verificar monotonía del índice
es_monotono_tel = df_tel.index.is_monotonic_increasing #Comprobacion de la monotonia: Los indices temporales avanzan correctamente de menor a mayor

print(f"Duplicados eliminados: {duplicados_antes_tel}")
print(f"Índice monótono: {es_monotono_tel}")

# Verificar si hay duplicados en el índice temporal
duplicados_index = df_tel.index.duplicated().sum()
if duplicados_index > 0:
    print(f"Hay {duplicados_index} timestamps duplicados en el índice")
    df_tel = df_tel[~df_tel.index.duplicated(keep='first')]
else:
    print("No hay timestamps duplicados")


Telemetria:
Duplicados eliminados: 0
Índice monótono: True
No hay timestamps duplicados


    2.2 Eventos

In [30]:
print("Eventos:")
# Eliminar duplicados exactos en telemetría
dup_evt = df_evt.duplicated().sum()
df_evt = df_evt[~df_evt.duplicated(keep='first')]

# Ordenar por tiempo de inicio (y fin como desempate)
df_evt = df_evt.sort_values(['ts_ini', 'ts_fin']).reset_index(drop=True)

# Chequeos
es_monotono_evt = df_evt['ts_ini'].is_monotonic_increasing
neg_dur = (df_evt['ts_fin'] < df_evt['ts_ini']).sum()
dup_ts_ini = df_evt['ts_ini'].duplicated().sum()

print(f"Duplicados eliminados: {dup_evt}")
print(f"Orden por ts_ini monótono: {es_monotono_evt}")
print(f"Eventos con ts_fin < ts_ini: {neg_dur}")
print(f"Timestamps ts_ini duplicados: {dup_ts_ini}")


Eventos:
Duplicados eliminados: 0
Orden por ts_ini monótono: True
Eventos con ts_fin < ts_ini: 0
Timestamps ts_ini duplicados: 0


    2.3 Botellas

In [31]:
print("Botellas:")
dup_pz = df_pz.duplicated().sum()
df_pz = df_pz[~df_pz.duplicated(keep='first')]

# Ordenar por ts_ciclo
df_pz = df_pz.sort_values('ts_ciclo').reset_index(drop=True)

# Chequeos
es_monotono_pz = df_pz['ts_ciclo'].is_monotonic_increasing
dup_ts_ciclo = df_pz['ts_ciclo'].duplicated().sum()
dup_id_botella = df_pz['id_botella'].duplicated().sum()

print(f"Duplicados eliminados: {dup_pz}")
print(f"Orden por ts_ciclo monótono: {es_monotono_pz}")
print(f"Timestamps ts_ciclo duplicados: {dup_ts_ciclo}")
print(f"id_botella duplicados: {dup_id_botella}")

Botellas:
Duplicados eliminados: 0
Orden por ts_ciclo monótono: True
Timestamps ts_ciclo duplicados: 0
id_botella duplicados: 0


PASO 3: Validaciones de rango

    3.1 Marcar valores fuera de rango (sin eliminar)

In [32]:
RANGO_TEMP = (18.0,35.0)
RANGO_VEL = (0.0,0.5)
RANGO_CAUDAL = (0.0,12.0)

df_tel['fuera_RANGO_TEMP'] = (df_tel['temp_prod'] < RANGO_TEMP[0]) | (df_tel['temp_prod'] > RANGO_TEMP[1])
df_tel['fuera_RANGO_VEL'] = (df_tel['vel_cinta'] < RANGO_VEL[0]) | (df_tel['vel_cinta'] > RANGO_VEL[1])
df_tel['fuera_RANGO_CAUDAL'] = (df_tel['caudal'] < RANGO_CAUDAL[0]) | (df_tel['caudal'] > RANGO_CAUDAL[1])

n_temp_fuera = df_tel['fuera_RANGO_TEMP'].sum()
n_vel_fuera = df_tel['fuera_RANGO_VEL'].sum()
n_caudal_fuera = df_tel['fuera_RANGO_CAUDAL'].sum()

print("="*60)
print("VALIDACIÓN DE RANGOS")
print("="*60)
print(f"temp_prod fuera de [{RANGO_TEMP[0]}, {RANGO_TEMP[1]}] °C: {n_temp_fuera} ({n_temp_fuera/len(df_tel)*100:.2f}%)")
print(f"vel_cinta fuera de [{RANGO_VEL[0]}, {RANGO_VEL[1]}] m/s: {n_vel_fuera} ({n_vel_fuera/len(df_tel)*100:.2f}%)")
print(f"caudal fuera de [{RANGO_CAUDAL[0]}, {RANGO_CAUDAL[1]}] ml/s: {n_caudal_fuera} ({n_caudal_fuera/len(df_tel)*100:.2f}%)")

# Mostrar estadísticas descriptivas
print("\nEstadísticas descriptivas:")
print(df_tel[['temp_prod', 'vel_cinta', 'caudal', 'energia_kwh']].describe())

VALIDACIÓN DE RANGOS
temp_prod fuera de [18.0, 35.0] °C: 0 (0.00%)
vel_cinta fuera de [0.0, 0.5] m/s: 0 (0.00%)
caudal fuera de [0.0, 12.0] ml/s: 0 (0.00%)

Estadísticas descriptivas:
           temp_prod      vel_cinta         caudal    energia_kwh
count  129601.000000  129601.000000  129601.000000  129601.000000
mean       25.646635       0.269995       7.735915      64.876601
std         2.545803       0.092667       2.532649      37.845824
min        18.000000       0.000000       0.000000       0.000000
25%        23.802999       0.246000       7.443000      31.746546
50%        25.648001       0.284000       8.321000      64.762093
75%        27.365999       0.330000       9.101000      97.557688
max        33.855000       0.380000      11.723000     130.067508


    3.2 Validar que energia_kwh no decrece (salvo cuantización)

In [33]:
# Calcular diferencias entre valores consecutivos de energía
df_tel['delta_energia'] = df_tel['energia_kwh'].diff()

# Contar cuántas veces la energía DECRECE (delta < 0)
# Nota: diff() genera NaN en la primera fila, lo ignoramos
decrementos = (df_tel['delta_energia'] < 0).sum() 
total_cambios = df_tel['delta_energia'].notna().sum() #Va a contar cuantas veces decrece

print("="*60)
print("VALIDACIÓN DE ENERGÍA NO DECRECIENTE")
print("="*60)
print(f"Total de cambios: {total_cambios:,}")
print(f"Decrementos detectados: {decrementos} ({decrementos/total_cambios*100:.3f}%)")

# Mostrar algunos ejemplos de decrementos (si existen)
if decrementos > 0:
    print("\n Ejemplos de energía que decrece:")
    ejemplos_decremento = df_tel[df_tel['delta_energia'] < 0][['energia_kwh', 'delta_energia']].head(10)
    print(ejemplos_decremento)
    
    # Estadísticas de los decrementos
    print("\n Estadísticas de los decrementos:")
    print(df_tel[df_tel['delta_energia'] < 0]['delta_energia'].describe())
else:
    print("\n No se detectaron decrementos en energia_kwh")

VALIDACIÓN DE ENERGÍA NO DECRECIENTE
Total de cambios: 129,600
Decrementos detectados: 0 (0.000%)

 No se detectaron decrementos en energia_kwh


PASO 4: Monotonicidad de energía (corrección de decrementos)

In [34]:
# PASO 4: Corregir decrementos de energía (si los hubiera)
# Guardamos la columna original para comparación
df_tel['energia_kwh_original'] = df_tel['energia_kwh'].copy()

# Identificar decrementos
decrementos_mask = df_tel['delta_energia'] < 0
n_correcciones = decrementos_mask.sum()

if n_correcciones > 0:
    print(f"Se encontraron {n_correcciones} decrementos. Corrigiendo...")
    
    # Hacer clip de deltas negativos a 0
    df_tel['delta_energia_corregida'] = df_tel['delta_energia'].clip(lower=0)
    
    # Reconstruir energía acumulada desde el primer valor
    energia_inicial = df_tel['energia_kwh'].iloc[0]
    df_tel['energia_kwh'] = energia_inicial + df_tel['delta_energia_corregida'].fillna(0).cumsum()
    
    # Recalcular delta_energia con valores corregidos
    df_tel['delta_energia'] = df_tel['energia_kwh'].diff()
    
    print(f"{n_correcciones} correcciones aplicadas")
else:
    print("No se requieren correcciones en energia_kwh")
    print("La señal es naturalmente monótona creciente")

# Verificación final
decrementos_final = (df_tel['delta_energia'] < 0).sum()
print(f"\nVerificación final: {decrementos_final} decrementos después de corrección")

No se requieren correcciones en energia_kwh
La señal es naturalmente monótona creciente

Verificación final: 0 decrementos después de corrección


PASO 5: Frecuencia y huecos temporales

    5.1 Confirmar frecuencia nominal de 1 Hz

In [35]:
# 5.1 Analizar la frecuencia de muestreo
print("="*60)
print("ANÁLISIS DE FRECUENCIA DE MUESTREO")
print("="*60)

# Calcular diferencias de tiempo entre muestras consecutivas
time_diffs = df_tel.index.to_series().diff()

# Contar muestras con intervalo de 1 segundo
intervalo_1s = time_diffs == pd.Timedelta(seconds=1)
n_1s = intervalo_1s.sum()
total = len(time_diffs) - 1  # -1 porque el primer valor es NaN

print(f"\nMuestras con intervalo de 1s: {n_1s}/{total}")

# Identificar huecos (intervalos > 1s)
huecos = time_diffs[time_diffs > pd.Timedelta(seconds=1)]
n_huecos = len(huecos)

print(f"\nHuecos detectados (intervalos > 1s): {n_huecos}")

if n_huecos > 0:
    print(f"\nEstadísticas de los huecos:")
    print(huecos.describe())
    
    # Clasificar huecos
    huecos_pequenos = huecos[huecos <= pd.Timedelta(seconds=10)]
    huecos_grandes = huecos[huecos > pd.Timedelta(seconds=10)]
    
    print(f"\nHuecos pequeños (≤10s): {len(huecos_pequenos)}")
    print(f"Huecos grandes (>10s): {len(huecos_grandes)}")
else:
    print(f"El numero de huecos es: {n_huecos}")
    print("La frecuencia nominal es de 1 Hz. Todos los saltos son de un segundo")
    print("No es necesario interpolar ni marcar segmentos invalidos")


ANÁLISIS DE FRECUENCIA DE MUESTREO

Muestras con intervalo de 1s: 129600/129600

Huecos detectados (intervalos > 1s): 0
El numero de huecos es: 0
La frecuencia nominal es de 1 Hz. Todos los saltos son de un segundo
No es necesario interpolar ni marcar segmentos invalidos


    5.2 Reindexar a rejilla de 1 segundo

In [36]:
# 5.2 Reindexar a rejilla regular de 1 segundo
print("\n" + "="*60)
print("REINDEXACIÓN A REJILLA DE 1 SEGUNDO")
print("="*60)

# Crear rejilla temporal de 1s desde el primer al último timestamp
ts_inicio = df_tel.index.min()
ts_fin = df_tel.index.max()
rejilla_1s = pd.date_range(start=ts_inicio, end=ts_fin, freq='1s')

print(f"\nRango temporal:")
print(f"   Inicio: {ts_inicio}")
print(f"   Fin: {ts_fin}")
print(f"   Duración: {ts_fin - ts_inicio}")

print(f"\nTamaño de los datos:")
print(f"   Muestras originales: {len(df_tel):,}")
print(f"   Rejilla de 1s: {len(rejilla_1s):,}")
print(f"   Diferencia (huecos): {len(rejilla_1s) - len(df_tel):,}")

# Reindexar el DataFrame a la rejilla de 1s
df_tel = df_tel.reindex(rejilla_1s)

print(f"\nDataFrame reindexado")


REINDEXACIÓN A REJILLA DE 1 SEGUNDO

Rango temporal:
   Inicio: 2025-02-12 08:00:00+00:00
   Fin: 2025-02-13 20:00:00+00:00
   Duración: 1 days 12:00:00

Tamaño de los datos:
   Muestras originales: 129,601
   Rejilla de 1s: 129,601
   Diferencia (huecos): 0

DataFrame reindexado


5.3 Rellenar huecos pequeños (≤10s) con interpolación

In [37]:
# 5.3 Rellenar huecos ≤ 10s
print("\n" + "="*60)
print("RELLENADO DE HUECOS PEQUEÑOS (≤10s)")
print("="*60)

# Identificar bloques de NaN consecutivos
df_tel['es_nan'] = df_tel['temp_prod'].isna()
df_tel['bloque_nan'] = (df_tel['es_nan'] != df_tel['es_nan'].shift()).cumsum()

# Calcular tamaño de cada bloque de NaN
tamano_bloques = df_tel[df_tel['es_nan']].groupby('bloque_nan').size()

# Clasificar bloques
bloques_pequenos = tamano_bloques[tamano_bloques <= 10]
bloques_grandes = tamano_bloques[tamano_bloques > 10]

print(f"\nBloques de NaN detectados:")
print(f"   Total de bloques: {len(tamano_bloques)}")
print(f"   Bloques ≤10s: {len(bloques_pequenos)} (se interpolarán)")
print(f"   Bloques >10s: {len(bloques_grandes)} (se marcarán como inválidos)")

# Crear máscara para huecos pequeños (≤10s)
mask_huecos_pequenos = df_tel['bloque_nan'].isin(bloques_pequenos.index) & df_tel['es_nan']

# Interpolación lineal para temp_prod y caudal en huecos pequeños
print(f"\nInterpolando temp_prod y caudal...")
df_tel.loc[mask_huecos_pequenos, 'temp_prod'] = df_tel['temp_prod'].interpolate(method='linear', limit=10)
df_tel.loc[mask_huecos_pequenos, 'caudal'] = df_tel['caudal'].interpolate(method='linear', limit=10)

# Forward-fill para vel_cinta (propagar último valor válido)
print(f"Forward-fill en vel_cinta...")
df_tel.loc[mask_huecos_pequenos, 'vel_cinta'] = df_tel['vel_cinta'].ffill(limit=10)



RELLENADO DE HUECOS PEQUEÑOS (≤10s)

Bloques de NaN detectados:
   Total de bloques: 0
   Bloques ≤10s: 0 (se interpolarán)
   Bloques >10s: 0 (se marcarán como inválidos)

Interpolando temp_prod y caudal...
Forward-fill en vel_cinta...


    5.4 Marcar huecos grandes (>10s) como inválidos

In [38]:
# 5.4 Marcar huecos grandes como inválidos
print("\n" + "="*60)
print("MARCADO DE HUECOS GRANDES (>10s)")
print("="*60)

# Crear columna para marcar segmentos inválidos
mask_huecos_grandes = df_tel['bloque_nan'].isin(bloques_grandes.index) & df_tel['es_nan']
df_tel['segmento_invalido'] = mask_huecos_grandes

# Contar segundos marcados como inválidos
n_invalidos = df_tel['segmento_invalido'].sum()
total_segundos = len(df_tel)

print(f"\nSegmentos marcados como inválidos:")
print(f"   Total de segundos inválidos: {n_invalidos:,}")
print(f"   Porcentaje: {n_invalidos/total_segundos*100:.2f}%")

if len(bloques_grandes) > 0:
    print(f"\nDetalle de huecos grandes:")
    for i, (bloque_id, tamano) in enumerate(bloques_grandes.items(), 1):
        inicio_hueco = df_tel[df_tel['bloque_nan'] == bloque_id].index.min()
        fin_hueco = df_tel[df_tel['bloque_nan'] == bloque_id].index.max()
        print(f"   Hueco {i}: {tamano}s desde {inicio_hueco} hasta {fin_hueco}")
        if i >= 5:
            print(f"   ... y {len(bloques_grandes)-5} huecos más")
            break

# Limpiar columnas auxiliares
df_tel = df_tel.drop(columns=['es_nan', 'bloque_nan'])

print(f"\n✅ Proceso de huecos completado")


MARCADO DE HUECOS GRANDES (>10s)

Segmentos marcados como inválidos:
   Total de segundos inválidos: 0
   Porcentaje: 0.00%

✅ Proceso de huecos completado


PASO 6: Detección de atípicos

    6.1 Detección por z-score (umbral ±3)

In [39]:
# 6.1 Detección de atípicos por z-score
print("="*60)
print("DETECCIÓN DE ATÍPICOS POR Z-SCORE")
print("="*60)

# Umbral estándar: valores con |z-score| > 3 son atípicos
UMBRAL_Z = 3

# Calcular z-score para cada variable
# z-score = (valor - media) / desviación estándar
df_tel['z_temp'] = (df_tel['temp_prod'] - df_tel['temp_prod'].mean()) / df_tel['temp_prod'].std()
df_tel['z_vel'] = (df_tel['vel_cinta'] - df_tel['vel_cinta'].mean()) / df_tel['vel_cinta'].std()
df_tel['z_caudal'] = (df_tel['caudal'] - df_tel['caudal'].mean()) / df_tel['caudal'].std()

# Marcar atípicos (|z| > 3)
df_tel['atipico_z_temp'] = df_tel['z_temp'].abs() > UMBRAL_Z
df_tel['atipico_z_vel'] = df_tel['z_vel'].abs() > UMBRAL_Z
df_tel['atipico_z_caudal'] = df_tel['z_caudal'].abs() > UMBRAL_Z

# Contar atípicos detectados
n_atip_temp = df_tel['atipico_z_temp'].sum()
n_atip_vel = df_tel['atipico_z_vel'].sum()
n_atip_caudal = df_tel['atipico_z_caudal'].sum()

print(f"\nAtípicos detectados (|z-score| > {UMBRAL_Z}):")
print(f"   temp_prod: {n_atip_temp} ({n_atip_temp/len(df_tel)*100:.3f}%)")
print(f"   vel_cinta: {n_atip_vel} ({n_atip_vel/len(df_tel)*100:.3f}%)")
print(f"   caudal: {n_atip_caudal} ({n_atip_caudal/len(df_tel)*100:.3f}%)")

# Mostrar ejemplos si existen
if n_atip_temp > 0:
    print("\nEjemplos de atípicos en temp_prod:")
    print(df_tel[df_tel['atipico_z_temp']][['temp_prod', 'z_temp']].head())

DETECCIÓN DE ATÍPICOS POR Z-SCORE

Atípicos detectados (|z-score| > 3):
   temp_prod: 43 (0.033%)
   vel_cinta: 0 (0.000%)
   caudal: 10833 (8.359%)

Ejemplos de atípicos en temp_prod:
                           temp_prod    z_temp
2025-02-13 03:59:56+00:00       18.0 -3.003624
2025-02-13 03:59:57+00:00       18.0 -3.003624
2025-02-13 04:00:03+00:00       18.0 -3.003624
2025-02-13 04:00:04+00:00       18.0 -3.003624
2025-02-13 04:00:05+00:00       18.0 -3.003624


    6.2 Detección por IQR (rango intercuartílico)

In [40]:
# 6.2 Detección de atípicos por IQR
print("\n" + "="*60)
print("DETECCIÓN DE ATÍPICOS POR IQR")
print("="*60)

# Calcular cuartiles y rango intercuartílico (IQR)
# IQR = Q3 - Q1
# Límites: [Q1 - 1.5*IQR, Q3 + 1.5*IQR]

for var in ['temp_prod', 'vel_cinta', 'caudal']:
    Q1 = df_tel[var].quantile(0.25)
    Q3 = df_tel[var].quantile(0.75)
    IQR = Q3 - Q1
    
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    # Marcar atípicos
    col_name = f'atipico_iqr_{var.split("_")[0]}'  # atipico_iqr_temp, atipico_iqr_vel, atipico_iqr_caudal
    df_tel[col_name] = (df_tel[var] < limite_inferior) | (df_tel[var] > limite_superior)
    
    n_atipicos = df_tel[col_name].sum()
    
    print(f"\n{var}:")
    print(f"   Q1: {Q1:.3f}")
    print(f"   Q3: {Q3:.3f}")
    print(f"   IQR: {IQR:.3f}")
    print(f"   Límites: [{limite_inferior:.3f}, {limite_superior:.3f}]")
    print(f"   Atípicos: {n_atipicos} ({n_atipicos/len(df_tel)*100:.3f}%)")


DETECCIÓN DE ATÍPICOS POR IQR

temp_prod:
   Q1: 23.803
   Q3: 27.366
   IQR: 3.563
   Límites: [18.458, 32.710]
   Atípicos: 125 (0.096%)

vel_cinta:
   Q1: 0.246
   Q3: 0.330
   IQR: 0.084
   Límites: [0.120, 0.456]
   Atípicos: 10833 (8.359%)

caudal:
   Q1: 7.443
   Q3: 9.101
   IQR: 1.658
   Límites: [4.956, 11.588]
   Atípicos: 10834 (8.360%)


    6.3 Consolidar marcas de atípicos

In [41]:
# 6.3 Consolidar detección de atípicos
print("\n" + "="*60)
print("CONSOLIDACIÓN DE ATÍPICOS")
print("="*60)

# Crear columna que marca si hay algún atípico (OR lógico)
# Un registro es atípico si al menos una variable lo es (por cualquier método)
df_tel['es_atipico'] = (
    df_tel['atipico_z_temp'] | df_tel['atipico_z_vel'] | df_tel['atipico_z_caudal'] |
    df_tel['atipico_iqr_temp'] | df_tel['atipico_iqr_vel'] | df_tel['atipico_iqr_caudal']
)

total_atipicos = df_tel['es_atipico'].sum()
porcentaje = total_atipicos / len(df_tel) * 100

print(f"\nRegistros con al menos un valor atípico:")
print(f"   Total: {total_atipicos:,}")
print(f"   Porcentaje: {porcentaje:.2f}%")

# Resumen por método
print(f"\nComparación de métodos:")
print(f"   Z-score: {(df_tel['atipico_z_temp'] | df_tel['atipico_z_vel'] | df_tel['atipico_z_caudal']).sum():,}")
print(f"   IQR: {(df_tel['atipico_iqr_temp'] | df_tel['atipico_iqr_vel'] | df_tel['atipico_iqr_caudal']).sum():,}")


CONSOLIDACIÓN DE ATÍPICOS

Registros con al menos un valor atípico:
   Total: 10,959
   Porcentaje: 8.46%

Comparación de métodos:
   Z-score: 10,876
   IQR: 10,959


PASO 7: Etiqueta RUN/STOP por segundo

    7.1 Construir máscara de paradas desde eventos

In [42]:
# 7.1 Construir máscara STOP_evt desde eventos.csv
print("="*60)
print("CONSTRUCCIÓN DE MÁSCARA RUN/STOP")
print("="*60)

# Filtrar eventos que implican parada
eventos_parada = df_evt[df_evt['tipo'].isin(['micro_parada', 'cambio_formato', 'limpieza'])].copy()

print(f"\nEventos de parada encontrados:")
print(f"   Total: {len(eventos_parada)}")
print(f"   micro_parada: {(eventos_parada['tipo'] == 'micro_parada').sum()}")
print(f"   cambio_formato: {(eventos_parada['tipo'] == 'cambio_formato').sum()}")
print(f"   limpieza: {(eventos_parada['tipo'] == 'limpieza').sum()}")

# Inicializar columna STOP_evt en False (por defecto está en marcha)
df_tel['STOP_evt'] = False

# Marcar como True los segundos que caen en intervalos [ts_ini, ts_fin)
for idx, evento in eventos_parada.iterrows():
    mascara_tiempo = (df_tel.index >= evento['ts_ini']) & (df_tel.index < evento['ts_fin'])
    df_tel.loc[mascara_tiempo, 'STOP_evt'] = True

n_stop_evt = df_tel['STOP_evt'].sum()
print(f"\nSegundos marcados como STOP por eventos: {n_stop_evt:,} ({n_stop_evt/len(df_tel)*100:.2f}%)")

CONSTRUCCIÓN DE MÁSCARA RUN/STOP

Eventos de parada encontrados:
   Total: 78
   micro_parada: 71
   cambio_formato: 5
   limpieza: 2



Segundos marcados como STOP por eventos: 10,755 (8.30%)


7.2 Definir RUN basado en velocidad de cinta

In [43]:
# 7.2 Definir RUN_vel basado en velocidad de cinta
print("\n" + "="*60)
print("DEFINICIÓN DE RUN_vel")
print("="*60)

# Umbral de velocidad para considerar que la máquina está en marcha
UMBRAL_VEL_RUN = 0.05  # m/s

# RUN_vel = True si vel_cinta >= 0.05 m/s
df_tel['RUN_vel'] = df_tel['vel_cinta'] >= UMBRAL_VEL_RUN

n_run_vel = df_tel['RUN_vel'].sum()
print(f"\nUmbral de velocidad: {UMBRAL_VEL_RUN} m/s")
print(f"Segundos con RUN_vel=True: {n_run_vel:,}")
print(f"Segundos con RUN_vel=False: {len(df_tel)-n_run_vel:,}")


DEFINICIÓN DE RUN_vel

Umbral de velocidad: 0.05 m/s
Segundos con RUN_vel=True: 118,768
Segundos con RUN_vel=False: 10,833


7.3 Combinar en estado final (RUN/STOP)

In [44]:
# 7.3 Definir estado final: RUN si RUN_vel=True Y STOP_evt=False
print("\n" + "="*60)
print("COMBINACIÓN DE CONDICIONES")
print("="*60)

# estado = RUN si (RUN_vel AND NOT STOP_evt), STOP en otro caso
df_tel['estado'] = 'STOP'
df_tel.loc[df_tel['RUN_vel'] & ~df_tel['STOP_evt'], 'estado'] = 'RUN'

# Convertir a tipo category para ahorrar memoria
df_tel['estado'] = df_tel['estado'].astype('category')

# Contar estados
n_run = (df_tel['estado'] == 'RUN').sum()
n_stop = (df_tel['estado'] == 'STOP').sum()

print(f"\nDistribución de estados:")
print(f"   RUN: {n_run:,}")
print(f"   STOP: {n_stop:,}")

# Análisis de transiciones
df_tel['cambio_estado'] = df_tel['estado'] != df_tel['estado'].shift()
n_transiciones = df_tel['cambio_estado'].sum() - 1  # -1 para excluir el primer valor

print(f"\nTransiciones de estado detectadas: {n_transiciones}")


COMBINACIÓN DE CONDICIONES

Distribución de estados:
   RUN: 118,768
   STOP: 10,833

Transiciones de estado detectadas: 151


PASO 8: Agregación a 1 minuto (diagnóstico temprano)

In [45]:
# PASO 8: Agregación temporal a 1 minuto
print("="*60)
print("AGREGACIÓN A 1 MINUTO")
print("="*60)

# Crear agregaciones por minuto
df_1min = df_tel.resample('1min').agg({
    'temp_prod': ['mean', lambda x: x.quantile(0.95)],
    'caudal': 'mean',
    'vel_cinta': 'mean',
    'energia_kwh': 'last',  # Último valor del minuto (acumulado)
    'estado': lambda x: (x == 'STOP').sum()  # Contar segundos en STOP
}).round(3)

# Aplanar nombres de columnas
df_1min.columns = ['temp_mean', 'temp_p95', 'caudal_mean', 'vel_cinta_mean', 'energia_kwh', 'segundos_stop']

# Calcular métricas derivadas
df_1min['pct_stop'] = (df_1min['segundos_stop'] / 60 * 100).round(2)
df_1min['segundos_run'] = 60 - df_1min['segundos_stop']
df_1min['pct_run'] = (df_1min['segundos_run'] / 60 * 100).round(2)

# Calcular delta de energía por minuto
df_1min['delta_energia_min'] = df_1min['energia_kwh'].diff()

# Información del resultado
print(f"\nDataFrame agregado:")
print(f"   Registros originales (1s): {len(df_tel):,}")
print(f"   Registros agregados (1min): {len(df_1min):,}")
print(f"   Rango temporal: {df_1min.index.min()} a {df_1min.index.max()}")

print(f"\nColumnas creadas:")
for col in df_1min.columns:
    print(f"   - {col}")

print(f"\nEstadísticas de disponibilidad:")
print(f"   Media % RUN por minuto: {df_1min['pct_run'].mean():.2f}%")
print(f"   Media % STOP por minuto: {df_1min['pct_stop'].mean():.2f}%")
print(f"   Minutos con 100% RUN: {(df_1min['pct_run'] == 100).sum()} ({(df_1min['pct_run'] == 100).sum()/len(df_1min)*100:.2f}%)")
print(f"   Minutos con 100% STOP: {(df_1min['pct_stop'] == 100).sum()} ({(df_1min['pct_stop'] == 100).sum()/len(df_1min)*100:.2f}%)")

print(f"\nPrimeros registros:")
print(df_1min.head())

AGREGACIÓN A 1 MINUTO



DataFrame agregado:
   Registros originales (1s): 129,601
   Registros agregados (1min): 2,161
   Rango temporal: 2025-02-12 08:00:00+00:00 a 2025-02-13 20:00:00+00:00

Columnas creadas:
   - temp_mean
   - temp_p95
   - caudal_mean
   - vel_cinta_mean
   - energia_kwh
   - segundos_stop
   - pct_stop
   - segundos_run
   - pct_run
   - delta_energia_min

Estadísticas de disponibilidad:
   Media % RUN por minuto: 91.65%
   Media % STOP por minuto: 8.35%
   Minutos con 100% RUN: 1910 (88.39%)
   Minutos con 100% STOP: 120 (5.55%)

Primeros registros:
                           temp_mean  temp_p95  caudal_mean  vel_cinta_mean  \
2025-02-12 08:00:00+00:00  24.997999    25.672          0.0             0.0   
2025-02-12 08:01:00+00:00  24.575001    24.756          0.0             0.0   
2025-02-12 08:02:00+00:00  24.604000    24.876          0.0             0.0   
2025-02-12 08:03:00+00:00  25.184000    25.523          0.0             0.0   
2025-02-12 08:04:00+00:00  25.219000    25.513  

# FASE 2: Ingeniería de variables y KPIs (NumPy + Pandas)

PASO 1: Cálculo de potencia instantánea desde energía acumulada

    Fórmula: P_kW = ΔE / Δt (donde Δt está en horas)

### Fórmulas implementadas

**Cálculo de potencia instantánea:**

$$\Delta E_i = \max\{E_i - E_{i-1}, 0\}$$

$$\Delta t_i = \frac{t_i - t_{i-1}}{3600} \text{ (horas)}$$

$$P_{\text{kW},i} = \frac{\Delta E_i}{\Delta t_i}$$

$$P_{\text{W},i} = 1000 \cdot P_{\text{kW},i}$$

**Suavizado opcional (media móvil):**
- Ventana: 5 segundos (centrada)
- Objetivo: Mitigar efectos de cuantización del contador de energía

In [46]:
# PASO 9: Calcular potencia instantánea
print("="*60)
print("CÁLCULO DE POTENCIA INSTANTÁNEA")
print("="*60)

# Calcular delta de energía (ya lo teníamos del Paso 4)
# df_tel['delta_energia'] ya existe

# Calcular delta de tiempo en horas
df_tel['delta_tiempo_h'] = df_tel.index.to_series().diff().dt.total_seconds() / 3600

# Calcular potencia en kW: P = ΔE / Δt
# Evitar división por cero
df_tel['P_kW'] = np.where(
    df_tel['delta_tiempo_h'] > 0,
    df_tel['delta_energia'] / df_tel['delta_tiempo_h'],
    0
)

# Suavizar potencia con media móvil de 5 segundos para mitigar cuantización
df_tel['P_kW_suavizada'] = df_tel['P_kW'].rolling(window=5, center=True, min_periods=1).mean()

# Mostrar resultados de las fórmulas aplicadas
print("\nPrimeros 10 valores calculados:")
print(df_tel[['energia_kwh', 'delta_energia', 'delta_tiempo_h', 'P_kW', 'P_kW_suavizada']].head(10))

print(f"\n✅ Potencia instantánea calculada")

CÁLCULO DE POTENCIA INSTANTÁNEA

Primeros 10 valores calculados:
                           energia_kwh  delta_energia  delta_tiempo_h    P_kW  \
2025-02-12 08:00:00+00:00     0.000000            NaN             NaN  0.0000   
2025-02-12 08:00:01+00:00     0.000336       0.000336        0.000278  1.2096   
2025-02-12 08:00:02+00:00     0.000625       0.000289        0.000278  1.0404   
2025-02-12 08:00:03+00:00     0.000943       0.000318        0.000278  1.1448   
2025-02-12 08:00:04+00:00     0.001175       0.000232        0.000278  0.8352   
2025-02-12 08:00:05+00:00     0.001420       0.000245        0.000278  0.8820   
2025-02-12 08:00:06+00:00     0.001709       0.000289        0.000278  1.0404   
2025-02-12 08:00:07+00:00     0.002075       0.000366        0.000278  1.3176   
2025-02-12 08:00:08+00:00     0.002328       0.000253        0.000278  0.9108   
2025-02-12 08:00:09+00:00     0.002617       0.000289        0.000278  1.0404   

                           P_kW_suavizada  

PASO 2: Agregación a 1 minuto (telemetría)

### Fórmulas de agregación

Para cada minuto $m$:

$$\text{temp\_mean}(m) = \text{mean}(T)$$

$$\text{temp\_p95}(m) = \text{p95}(T)$$

$$\text{caudal\_mean}(m) = \text{mean}(q)$$

$$\text{P\_kW\_mean}(m) = \text{mean}(P)$$

$$\%\text{STOP}(m) = 100 \cdot \frac{\#\{i \in m : \text{estado}_i = \text{STOP}\}}{60}$$

Estas series minuto servirán como base para KPIs horarios/por turno.

In [47]:
# PASO 10: Agregación a 1 minuto (telemetría)
print("="*60)
print("AGREGACIÓN A 1 MINUTO (TELEMETRÍA)")
print("="*60)

# Crear agregaciones por minuto aplicando las fórmulas
df_1min = df_tel.resample('1min').agg({
    'temp_prod': ['mean', lambda x: x.quantile(0.95)],
    'caudal': 'mean',
    'P_kW': 'mean',
    'estado': lambda x: (x == 'STOP').sum()
}).round(3)

# Aplanar nombres de columnas
df_1min.columns = ['temp_mean', 'temp_p95', 'caudal_mean', 'P_kW_mean', 'segundos_stop']

# Calcular %STOP
df_1min['pct_STOP'] = ((df_1min['segundos_stop'] / 60) * 100).round(2)

print(f"\nPrimeros registros:")
print(df_1min.head(10))

print(f"\n✅ Agregación a 1 minuto completada")

AGREGACIÓN A 1 MINUTO (TELEMETRÍA)

Primeros registros:
                           temp_mean  temp_p95  caudal_mean  P_kW_mean  \
2025-02-12 08:00:00+00:00  24.997999    25.672        0.000      1.104   
2025-02-12 08:01:00+00:00  24.575001    24.756        0.000      1.155   
2025-02-12 08:02:00+00:00  24.604000    24.876        0.000      1.141   
2025-02-12 08:03:00+00:00  25.184000    25.523        0.000      1.167   
2025-02-12 08:04:00+00:00  25.219000    25.513        0.000      1.208   
2025-02-12 08:05:00+00:00  25.625000    25.908        0.000      1.118   
2025-02-12 08:06:00+00:00  25.892000    26.101        0.000      1.199   
2025-02-12 08:07:00+00:00  26.014999    26.525        0.000      1.208   
2025-02-12 08:08:00+00:00  25.455000    25.874        8.709      3.881   
2025-02-12 08:09:00+00:00  24.711000    24.880        9.011      3.837   

                           segundos_stop  pct_STOP  
2025-02-12 08:00:00+00:00             60    100.00  
2025-02-12 08:01:00+00:

PASO 3: Clasificación de botellas por tolerancia de peso

### Objetivo de masa por formato

$$m_{\text{obj}}(250) = 250\text{ g}, \quad m_{\text{obj}}(500) = 500\text{ g}$$

### Criterio de tolerancia

Con tolerancia típica del ±2%, una unidad está dentro de tolerancia si:

$$|\text{peso\_lleno\_g} - m_{\text{obj}}(f)| \leq 0.02 \cdot m_{\text{obj}}(f)$$

Donde:
- $\text{peso\_lleno\_g}$ es el peso neto de la botella (en gramos)
- $f$ es el formato (250 o 500)
- $m_{\text{obj}}(f)$ es la masa objetivo según el formato

In [ ]:
# PASO 11: Clasificación de botellas por tolerancia de peso
print("="*60)
print("CLASIFICACIÓN DE BOTELLAS POR TOLERANCIA")
print("="*60)

# Definir masa objetivo según formato
MASA_OBJ = {250: 250.0, 500: 500.0}
TOLERANCIA = 0.02  # ±2%

# Crear columna con masa objetivo según el formato de cada botella
df_pz['masa_objetivo'] = df_pz['formato_ml'].map(MASA_OBJ)

# Calcular desviación absoluta respecto al objetivo
df_pz['desviacion_abs'] = np.abs(df_pz['peso_neto'] - df_pz['masa_objetivo'])

# Calcular límite de tolerancia (2% de la masa objetivo)
df_pz['limite_tolerancia'] = TOLERANCIA * df_pz['masa_objetivo']

# Clasificar: dentro_tolerancia = True si |peso_neto - m_obj| ≤ 0.02 * m_obj
df_pz['dentro_tolerancia'] = df_pz['desviacion_abs'] <= df_pz['limite_tolerancia']

# Mostrar resultados
print(f"\nPrimeros registros clasificados:")
print(df_pz[['id_botella', 'formato_ml', 'peso_neto', 'masa_objetivo', 'desviacion_abs', 'dentro_tolerancia']].head(10))

print(f"\n✅ Clasificación completada")

CLASIFICACIÓN DE BOTELLAS POR TOLERANCIA


KeyError: 'formato'

In [49]:
# PASO 11: Clasificación de botellas por tolerancia de peso
print("="*60)
print("CLASIFICACIÓN DE BOTELLAS POR TOLERANCIA")
print("="*60)

# Primero verificar las columnas disponibles
print(f"\nColumnas de df_pz:")
print(df_pz.columns.tolist())
print(f"\nPrimeras filas de df_pz:")
print(df_pz.head())

CLASIFICACIÓN DE BOTELLAS POR TOLERANCIA

Columnas de df_pz:
['ts_ciclo', 'id_botella', 'formato_ml', 'tiempo_ciclo_s', 'peso_lleno_g', 'ok_ng']

Primeras filas de df_pz:
                   ts_ciclo  id_botella  formato_ml  tiempo_ciclo_s  \
0 2025-02-12 08:08:03+00:00           1         500           483.0   
1 2025-02-12 08:08:06+00:00           2         500             3.0   
2 2025-02-12 08:08:09+00:00           3         500             3.0   
3 2025-02-12 08:08:12+00:00           4         500             3.0   
4 2025-02-12 08:08:15+00:00           5         500             3.0   

   peso_lleno_g ok_ng  
0        499.69    OK  
1        498.71    OK  
2        501.08    OK  
3        503.04    OK  
4        498.84    OK  
